In [32]:
import pandas as pd

In [159]:
def wrangle(path):
    temp = pd.read_csv(path)
    dataframe = (temp.
             loc[temp["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season', 'date']).
             reset_index().
             drop(columns = ['index'])
)
    return dataframe

In [160]:
wrangle("../../data/raw/mlb_elo_latest.csv")

,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,rating1_pre,rating2_pre,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,SEA,ANA,1493.538649,1492.763809,0.535594,0.464406,1489.919377,1496.383081,1482.869681,1500.492483,...,44.084974,39.929048,-19.659891,-44.827616,0.545272,0.454728,1480.094364,1503.267801,1.0,7.0
1,SDP,COL,1534.463116,1469.938927,0.624709,0.375291,1531.923574,1472.478469,1556.846373,1450.832758,...,48.061291,53.836186,-16.053037,2.460504,0.655171,0.344829,1554.814951,1452.864180,1.0,3.0
2,LAD,ARI,1584.632857,1438.785029,0.726656,0.273344,1585.624562,1437.793324,1595.910996,1443.884646,...,53.122349,52.166927,-10.727841,21.408305,0.695995,0.304005,1596.787439,1443.008203,7.0,4.0
3,SFG,WSN,1549.613822,1513.998828,0.584961,0.415039,1551.089689,1512.522961,1529.367363,1515.442810,...,59.656903,47.743591,34.283469,-16.077402,0.624387,0.375613,1530.332580,1514.477593,3.0,1.0
4,TEX,OAK,1465.150350,1530.314320,0.441036,0.558964,1463.355286,1532.109384,1458.196176,1519.456183,...,49.512599,56.874158,-6.635993,17.367272,0.412738,0.587262,1456.963359,1520.689000,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342,PHI,ATL,1499.854980,1537.334830,0.480611,0.519389,1501.313693,1535.876117,1506.197988,1533.503489,...,56.982418,53.435208,25.593166,-0.900219,0.537195,0.462805,1507.143856,1532.557621,3.0,2.0
1343,CHC,PIT,1510.606166,1466.885465,0.596242,0.403758,1508.311600,1469.180030,1506.271787,1433.526669,...,56.657868,47.268712,35.374576,-2.657647,0.687642,0.312358,1504.178744,1435.619713,3.0,5.0
1344,MIL,MIN,1502.386929,1522.960488,0.504931,0.495069,1503.802713,1521.544704,1517.724543,1537.681888,...,57.894228,57.914464,34.030150,17.163499,0.533019,0.466981,1518.710560,1536.695871,6.0,5.0
1345,DET,CLE,1450.437505,1513.164945,0.444496,0.555504,1451.992754,1511.609696,1446.323353,1513.291422,...,47.172259,61.318392,-0.833903,43.767481,0.378466,0.621534,1447.667412,1511.947362,3.0,2.0


In [85]:
def initial_correct(path):
    temp = pd.read_csv(path)
    df = (temp.
             loc[temp["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
         )
    correct1 = df.loc[((df['elo_prob1'] > df['elo_prob2']) & (df['score1'] > df['score2']))]
    correct2 = df.loc[((df['elo_prob1'] < df['elo_prob2']) & (df['score1'] < df['score2']))]
    correct_combine = [correct1, correct2]
    initial_correct = pd.concat(correct_combine).sort_values(by=['date']).reset_index()
    return initial_correct

In [90]:
initial_correct("../../data/raw/mlb_elo_latest.csv").shape

(773, 22)

In [89]:
def pitcher_correct(path):
    temp = pd.read_csv(path)
    df = (temp.
             loc[temp["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
         )
    temp1 = df.loc[((df['rating_prob1'] > df['rating_prob2']) & (df['score1'] > df['score2']))]    
    temp2 = df.loc[((df['rating_prob1'] < df['rating_prob2']) & (df['score1'] < df['score2']))]
    correct_combine = [temp1, temp2]
    pitcher_correct = pd.concat(correct_combine).sort_values(by=['date']).reset_index().drop(columns = ['pitcher1_rgs', 'pitcher2_rgs'])
    return pitcher_correct

In [91]:
pitcher_correct("../../data/raw/mlb_elo_latest.csv").shape

(773, 22)

In [92]:
def only_pitcher(path):
    temp = pd.read_csv(path)
    df = (temp.
             loc[temp["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
         )
    check1 = df.loc[((df['pitcher1_rgs'] < df['pitcher2_rgs']) & (df['score1'] < df['score2']))]   
    check2 = df.loc[((df['pitcher1_rgs'] > df['pitcher2_rgs']) & (df['score1'] > df['score2']))]
    correct_combine = [check1, check2]
    pitcher_correct = pd.concat(correct_combine).sort_values(by=['date']).reset_index().drop(columns = ['pitcher1_rgs', 'pitcher2_rgs'])
    return pitcher_correct

In [93]:
only_pitcher("../../data/raw/mlb_elo_latest.csv").shape

(741, 22)

In [123]:
def daily_correct(path):
    temp = pd.read_csv(path)
    df = (temp.
             loc[temp["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
         )
    temp1 = df.loc[((df['rating_prob1'] > df['rating_prob2']) & (df['score1'] > df['score2']))]    
    temp2 = df.loc[((df['rating_prob1'] < df['rating_prob2']) & (df['score1'] < df['score2']))]
    correct_combine = [temp1, temp2]
    pitcher_correct = pd.concat(correct_combine).sort_values(by=['date']).reset_index().drop(columns = ['pitcher1_rgs', 'pitcher2_rgs'])
   
    dates = pitcher_correct['date'].unique()
    counts = pitcher_correct['date'].value_counts()
    daily = pd.DataFrame(counts)
    daily = (daily.
                reset_index().
                sort_values(by=['index']).
                rename(columns = {'date' : 'Correct Guesses'}).
                rename(columns = {'index':'Date',}).
                sort_values(by=['Date'])
           )
    daily = (daily.
             rename(columns = {'date' : 'Correct Guesses'}).
             rename(columns = {'index':'Date',}).
             sort_values(by=['Date']).
             reset_index().
             drop(columns=['index'])
            )
    return daily

In [124]:
daily_correct("../../data/raw/mlb_elo_latest.csv")

,Date,Correct Guesses
0,2021-04-01,8
1,2021-04-02,4
2,2021-04-03,8
3,2021-04-04,7
4,2021-04-05,6
...,...,...
97,2021-07-07,10
98,2021-07-08,5
99,2021-07-09,11
100,2021-07-10,9


In [140]:
def find_team(path, team_name):
    temp = pd.read_csv(path)
    df = (temp.
             loc[temp["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
         )
          
    team_1 = df.loc[df['team1'] == team_name]
    team1 = team_1.rename(columns={'team1':'team', 'rating1_post':'rating'})
    team_2 = df.loc[df['team2'] == team_name]
    team2 = team_2.rename(columns={'team2':'team', 'rating2_post':'rating'})
                 
    home = team1[['date', 'team', 'rating']]
    away = team2[['date', 'team', 'rating']]
    team = pd.concat([home, away])
    team = team.sort_values(by=['date']).reset_index().drop(columns=['index'])

    return team

In [158]:
find_team("../../data/raw/mlb_elo_latest.csv", "ARI")

,date,team,rating
0,2021-04-01,ARI,1487.910868
1,2021-04-02,ARI,1486.959019
2,2021-04-03,ARI,1485.231401
3,2021-04-04,ARI,1487.127652
4,2021-04-06,ARI,1488.376528
...,...,...,...
87,2021-07-07,ARI,1446.225510
88,2021-07-08,ARI,1443.329965
89,2021-07-09,ARI,1446.087607
90,2021-07-10,ARI,1443.884646


In [148]:
list

array(['SEA', 'SDP', 'LAD', 'SFG', 'TEX', 'MIN', 'MIL', 'HOU', 'TBD',
       'NYM', 'FLA', 'BOS', 'BAL', 'CHC', 'CLE', 'ARI', 'ANA', 'KCR',
       'PIT', 'NYY', 'OAK', 'COL', 'ATL', 'DET', 'CIN', 'TOR', 'PHI',
       'WSN', 'CHW', 'STL'], dtype=object)

In [13]:
def correct(path):
    temp = pd.read_csv(path)
    df = (temp.
             loc[temp["date"] < '2021-08-12'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
)
    temp1 = df.loc[((df['rating_prob1'] > df['rating_prob2']) & (df['score1'] > df['score2']))]    
    temp2 = df.loc[((df['rating_prob1'] < df['rating_prob2']) & (df['score1'] < df['score2']))]
    correct_adj = pd.concat([temp1, temp2])
    dates = correct_adj['date'].unique()
    dicc = correct_adj['date'].value_counts()
    data = pd.DataFrame(dicc)
    data = data.reset_index()
    data.sort_values(by=['index'])
    right = data.rename(columns = {'date' : 'Correct Guesses'})
    right = right.rename(columns = {'index':'Date',})
    right = right.sort_values(by=['Date'])
    right = right.reset_index()
    right = right.drop(columns=['index'])
    
    return right

In [15]:
correct("../../data/raw/mlb_elo_latest.csv")

,Date,Correct Guesses
0,2021-04-01,8
1,2021-04-02,4
2,2021-04-03,8
3,2021-04-04,7
4,2021-04-05,6
...,...,...
97,2021-07-07,10
98,2021-07-08,5
99,2021-07-09,11
100,2021-07-10,9


In [64]:
def find_pitcher(path, pitcher):
    df = pd.read_csv(path)
    dataframe = (df.
             loc[df["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season']).
             reset_index().
             drop(columns = ['index'])
)
    temp1 = df.loc[df['pitcher1'] == pitcher]
    temp2 = df.loc[df['pitcher2'] == pitcher]
    
    sep1 = temp1[['date', 'pitcher1', 'pitcher1_rgs']]
    sep1 = sep1.rename(columns={'pitcher1':'pitcher', 'pitcher1_rgs':'rating'})
    sep2 = temp2[['date', 'pitcher2', 'pitcher2_rgs']]
    sep2 = sep2.rename(columns={'pitcher2':'pitcher', 'pitcher2_rgs':'rating'})
    
    sep = pd.concat([sep1, sep2])
    sep = sep.sort_values(by=['date']).reset_index().drop(columns=['index'])
    
    return sep

In [65]:
find_pitcher("../../data/raw/mlb_elo_latest.csv", "Clayton Kershaw")

,date,pitcher,rating
0,2021-04-01,Clayton Kershaw,57.256595
1,2021-04-06,Clayton Kershaw,56.383889
2,2021-04-11,Clayton Kershaw,57.301386
3,2021-04-17,Clayton Kershaw,57.865068
4,2021-04-23,Clayton Kershaw,58.572803
5,2021-04-28,Clayton Kershaw,58.167399
6,2021-05-04,Clayton Kershaw,58.929377
7,2021-05-08,Clayton Kershaw,57.472780
8,2021-05-14,Clayton Kershaw,58.094223
9,2021-05-19,Clayton Kershaw,57.543328
